In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import sys
sys.path.insert(0, '../../scripts/classifier')
from nets import net
from loader import dwellingsDataset

In [3]:
classifier = net().float()
optimizer = optim.Adam(classifier.parameters(), lr=1e-4)
dataset = dwellingsDataset()
data = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=4)

In [4]:
classifier.train()

net(
  (conv1): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (conv4): Conv2d(128, 2, kernel_size=(1, 1), stride=(1, 1))
  (logsoftmax): LogSoftmax()
)

In [5]:
optimizer.zero_grad()

In [6]:
for batch in data:
    x = batch[0].float()
    y = batch[1].long()
    break

In [7]:
print(x.shape)
print(y.shape)

torch.Size([100, 4, 128, 128])
torch.Size([100, 128, 128])


In [8]:
output = classifier(x)

In [9]:
output.shape

torch.Size([100, 2, 128, 128])

In [10]:
def loss_nll(predicts, targets):
    '''
    '''

    loss = nn.NLLLoss()
    output = loss(predicts, targets)

    return output

In [11]:
loss = loss_nll(output, y)

In [17]:
loss.item()

3.3977935314178467